The base url for all sandbox api endpoints is api.cert.tastyworks.com
The websocket url for account streamer updates is streamer.cert.tastyworks.com

In [ ]:
from tastytrade import Account, Session
import json
with open('session.json', 'r') as file:
    credentials = json.load(file)
    
session = Session(credentials['username'], credentials['password'], is_test=True) # is_test uses a sandbox account
accounts = await Account.a_get_accounts(session)

In [ ]:
if len(accounts) == 1:
    account = accounts[0]
    balance = account.get_balances(session)
    print(f"Loading single account: {balance.account_number} \t ${balance.cash_balance}")
    positions = account.get_positions(session)
    print(f"Total positions: {len(positions)}")

In [63]:
import tastytrade
import pandas as pd

In [ ]:
from tastytrade.dxfeed import Greeks
from tastytrade.instruments import get_option_chain
from tastytrade.utils import get_tasty_monthly

chain = get_option_chain(session, 'SPLG')
exp = get_tasty_monthly()  # 45 DTE expiration!
subs_list = [chain[exp][0].streamer_symbol]

# subs_list = [chain[exp][0]]
# print(subs_list)

async with tastytrade.DXLinkStreamer(session) as streamer:
    await streamer.subscribe(Greeks, subs_list)
    greeks = await streamer.get_event(Greeks)
    print(greeks)

In [57]:
from tastytrade import Session, DXLinkStreamer
from tastytrade.instruments import get_option_chain
from tastytrade.utils import get_tasty_monthly
from tastytrade.dxfeed import Greeks, Quote
import asyncio
from datetime import datetime, timedelta

from decimal import Decimal

async def get_underlying_price(session, symbol):
    async with DXLinkStreamer(session) as streamer:
        await streamer.subscribe(Quote, [symbol])
        quote = await streamer.get_event(Quote)
        if quote:
            return Decimal(str((quote.bidPrice + quote.askPrice) / 2))
    return None

async def get_option_ivr(session, symbol):
    chain = get_option_chain(session, symbol)
    exp = get_tasty_monthly()  # This gets the expiration closest to 45 DTE
    
    underlying_price = await get_underlying_price(session, symbol)
    if underlying_price is None:
        return None
    
    # Get the ATM (At The Money) option
    atm_option = min(chain[exp], key=lambda x: abs(x.strike_price - underlying_price))
    
    async with DXLinkStreamer(session) as streamer:
        await streamer.subscribe(Greeks, [atm_option.streamer_symbol])
        greeks = await streamer.get_event(Greeks)
        
        if greeks:
            return {
                'symbol': symbol,
                'expiration': exp,
                'strike': atm_option.strike_price,
                'ivr': greeks.volatility  # This is actually IV, not IVR
            }
        else:
            return None

In [80]:
stock_list = [
    'SPY',   # SPDR S&P 500 ETF
    'QQQ',   # Invesco QQQ Trust (tracks Nasdaq-100)
    'NVDA',  # Nvidia
    'TSLA',  # Tesla
    'AAPL',  # Apple
    'AMZN',  # Amazon
    'MSFT',  # Microsoft
    'AMD',   # Advanced Micro Devices
    'META',  # Meta Platforms (formerly Facebook)
    'GOOGL', # Alphabet (Google)
    'NFLX',  # Netflix
    'IWM',   # iShares Russell 2000 ETF
    'INTC',  # Intel
    'PLTR',  # Palantir Technologies
    'CRM',   # Salesforce
    'HUBS',  # HubSpot
    'GME',   # GameStop
    'CAT',   # Caterpillar
    'BA',    # Boeing
    'FXI',   # iShares China Large Cap ETF
    'LRCX',  # Lam Research
    'WHR',   # Whirlpool
    'TMUS',  # T-Mobile
    'ANET',  # Arista Networks
    'GE',    # General Electric
    'MS',    # Morgan Stanley
    'NVAX',  # Novavax
    'ACGL',  # Arch Capital Group
    'ETSY',  # Etsy
    'IRTC',  # iRhythm Technologies
    'BTI',    # British American Tobacco
    'T',
    'F',
]
tasks = [get_option_ivr(session, symbol) for symbol in stock_list]
results = await asyncio.gather(*tasks)

In [ ]:
# Get today's date
from datetime import datetime, date
today = date.today()

# Create a list of dictionaries from the results
data = [
    {
        "Symbol": result['symbol'],
        "Expiration": result['expiration'],
        "Days to Expiration": (result['expiration'] - today).days,
        "Strike": result['strike'],
        "IV": result['ivr'],
    }
    for result in results if result
]

# Create a DataFrame
df = pd.DataFrame(data)

# Filter the DataFrame to show only options with IV over 60%
df_filtered = df[df['IV'] > 0.55]

# Display the DataFrame as an interactive table
display(df_filtered)